# Продвинутый Python, лекция 10

**Лектор:** Петров Тимур

**Семинаристы:** Бузаев Федор, Дешеулин Олег, Коган Александра, Васина Олеся, Садуллаев Музаффар

На прошлых занятиях мы говорили про `Flask`, помните? Что ж... забудьте! Шутка. Сегодня мы рассмотрим более современный и свежий формат реализации веб-серверов. Будем изучать библиотеку `FastAPI`. Заодно поговорим про надежность эксплуатации в терминах клиент-серверной архитектуры.

**Пример.** Напишем сервер, у которого будет только одна ручка, через которую клиент может узнать, жив ли сервер.

**Примечание.** Запускайте код не в `Jupyter Notebook`, так как есть некоторые особенности реализации, не позволяющие это сделать.

In [ ]:
from http import HTTPStatus

import uvicorn
from fastapi import FastAPI, Response

app = FastAPI()


@app.get("/api/v1/healthcheck")
def healthcheck() -> Response:
    """Проверить, что сервер активен."""
    return Response(status_code=HTTPStatus.OK)


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

Главное изменение, которое можно заметить, это явное указание HTTP-метода еще на уровне процедуры. А еще есть какой-то `uvicorn`... Про него расскажем на семинаре. В остальном все максимально похоже на `Flask`.

Теперь откроем `http://localhost:8000/docs`. О, магия! Что-то симпатичное. Этот интерфейс называется `Swagger`. Если очень коротко и примтивно, то это инструмент, который позволяет на основе некоторой спецификации (формальное описание API) выполнять запросы к некоторому серверу от имени клиента.

`Swagger` строится на основе спецификации от `OpenAPI`. Давайте взглянем на спецификацию на примере нашего сервера. Перейдем в `Swagger`. Слева вверху можно увидеть гиперссылку на `/openapi.json` - перейдем по ней. Увидим следующее:

```json
{
  "openapi": "3.1.0",
  "info": {
    "title": "FastAPI",
    "version": "0.1.0"
  },
  "paths": {
    "/api/v1/healtcheck": {
      "get": {
        "summary": "Healthcheck",
        "description": "Проверить, что сервер активен.",
        "operationId": "healthcheck_api_v1_healtcheck_get",
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {

                }
              }
            }
          }
        }
      }
    }
  }
}
```

Как уже было сказано, это формализация методов нашего сервера. Мы описали, какие ручки существуют, что они могут вернуть и для чего они предназначены. Обратите внимание, что `FastAPI` подхватил название функции и ее докстринг прямо из кода. Круто ж! Это один из значимых плюсов `FastAPI`. Код становится самодокументируемым.

Давайте, на всякий случай, еще раз уточним:

* `Swagger` - инструмент (UI);
* `OpenAPI` - спецификация (документ).

Зачем вообще нужна `OpenAPI`-спецификация? Во-первых, чтобы генерировать `Swagger UI`. Это очевидно. Но это, к слову, не единственное применение. Давайте рассмотрим следующий пример.

**Пример.** Напишем сервер, который будет делать какую-то сложную вычислительную задачу и возвращать клиенту ее результат.

In [ ]:
from math import ceil, sqrt

import uvicorn
from fastapi import FastAPI

app = FastAPI()


def is_prime(n: int) -> bool:
    """Проверить, что число простое."""
    if n == 2:
        return True

    if n % 2 == 0:
        return False

    bound = ceil(sqrt(n))
    start, step = 3, 2

    return all(n % divisor != 0 for divisor in range(start, bound + 1, step))


def cpu(n: int) -> int:
    """Посчитать количество простых чисел от `2` до `n`."""
    return sum(is_prime(number) for number in range(2, n + 1))


@app.get("/api/v1/primes")
def count_primes(n: int) -> int:
    """Посчитать количество простых чисел от `2` до `n`."""
    return cpu(n)


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

Теперь откроем спецификацию:

```json
{
  "openapi": "3.1.0",
  "info": {
    "title": "FastAPI",
    "version": "0.1.0"
  },
  "paths": {
    "/api/v1/primes": {
      "get": {
        "summary": "Count Primes",
        "description": "Посчитать количество простых чисел от `2` до `n`.",
        "operationId": "count_primes_api_v1_primes_get",
        "parameters": [
          {
            "name": "n",
            "in": "query",
            "required": true,
            "schema": {
              "type": "integer",
              "title": "N"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {
                  "type": "integer",
                  "title": "Response Count Primes Api V1 Primes Get"
                }
              }
            }
          },
          "422": {
            "description": "Validation Error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/HTTPValidationError"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "HTTPValidationError": {
        "properties": {
          "detail": {
            "items": {
              "$ref": "#/components/schemas/ValidationError"
            },
            "type": "array",
            "title": "Detail"
          }
        },
        "type": "object",
        "title": "HTTPValidationError"
      },
      "ValidationError": {
        "properties": {
          "loc": {
            "items": {
              "anyOf": [
                {
                  "type": "string"
                },
                {
                  "type": "integer"
                }
              ]
            },
            "type": "array",
            "title": "Location"
          },
          "msg": {
            "type": "string",
            "title": "Message"
          },
          "type": {
            "type": "string",
            "title": "Error Type"
          }
        },
        "type": "object",
        "required": [
          "loc",
          "msg",
          "type"
        ],
        "title": "ValidationError"
      }
    }
  }
}
```

Обратите внимание, насколько выросло описание. Что изменилось? Появились входные и выходные типы. Причем они появились как для успешных ответов (`2XX`), так и для ошибочных (`4XX`). И все же, где наша выгода? Зачем нужна `OpenAPI`-спецификацию, кроме генерации `Swagger`? Наша выгода называется "автоклиент".

**Автоклиент** - это программа, которая по описанию API строит библиотечный интерфейс для взаимодействия с сервером, который удовлетворяет этой спецификации. Сгенерированная библиотека включает в себя модели ответов и методы для взаимодействия с сервером. Иногда автоклиентом называют саму сгенерированную библиотеку.

**Пример.** Автоклиент для сервера выше мог бы выглядеть следующим образом.

In [ ]:
def count_primes(n: int) -> int:
    """Посчитать количество простых чисел от `2` до `n`."""
    # 1. Установить соединение с сервером
    # 2. Выполнить запрос к серверу
    # 3. Дождаться ответа со стороны сервера
    # 4. Обработать ответ и вернуть пользователю

Тогда в коде это могло бы выглядеть вот так:

In [ ]:
from random import randint


def main() -> None:
    n = randint(2, 100500 + 1)

    # Неявный запрос на сервер
    count = count_primes(n)

    print(f"Между 2 и {n} существует {count} простыъ чисел")

Такой подход распространен в промышленной разработке в условиях микросервисной архитектуры. Вместо того, чтобы самостоятельно писать библиотеки под API каждого сервиса, достаточно скормить автоклиенту спецификацию. А он уже сам сгенерирует библиотеку, через которую можно взаимодействовать с API сервиса.

Это особенно полезно в условиях часто меняющихся (регулярно обновляющихся) API, так как не придется по десять раз переписывать собственный код. Для разработчиков это избавление от рутины, а для бизнеса дешевизна разработки, так как все происходит автоматически.

**Примечание.** Попробуйте передать НЕ число. `FastAPI` это отловит. Причем как на уровне `Swagger`, так и на уровне самого сервера. Для второго случая, чтобы убедиться, перейдите по ссылке: http://localhost:8000/api/v1/primes?n=bebra.

**Пример.** Напишем сервер, который будет возвращать клиенту список файлов в текущей директории сервера.

In [ ]:
from os import getcwd, listdir

import uvicorn
from fastapi import FastAPI

app = FastAPI()


@app.get("/api/v1/ls")
def ls() -> list[str]:
    """Вывести содержимое текущей директории [со стороны сервера]."""
    cwd = getcwd()
    return listdir(cwd)


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

**Примечание.** Помним, что вместо `os.getcwd` и `os.lisdir` лучше пользоваться `pathlib.Path.cwd` и `pathlib.Path.iterdir` соответственно.

**Вопрос.** Уважаемые знатоки, в чем проблема кода выше? Вспоминаем недавние занятия.

**Ответ.** Самые внимательные сразу заметят, что мы выполняем системный вызов. По сути мы передаем управление некоторому внешнему источнику (операционной системе), который явным образом не связан с нашей программой. Получается, мы блокируем основной поток до тех пор, пока не получим ответ от этой системы. Это значит, что сервер в течение всего времени системного будет обрабатывать буквально один запрос от одного пользователя, то есть для других пользователей наш сервер зависнет. Давайте вспомним одно из недавних занятий. Как избегают блокирующий ввод-вывод? Правильно, через асинхронность. Как Вы можете понять, `FastAPI` поддерживает асинхронность.

**Пример.** Перепишем сервер на асинхронный формат.

In [ ]:
from asyncio import to_thread
from os import getcwd, listdir

import uvicorn
from fastapi import FastAPI

app = FastAPI()


@app.get("/api/v1/ls")
async def ls() -> list[str]:
    """Вывести содержимое текущей директории [со стороны сервера]."""
    cwd = await to_thread(getcwd)
    return await to_thread(listdir, cwd)


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

Теперь нашему серверу не страшно зависание операционной системы, потому что ожиданием занимается отдельный поток (не основной). Использование `asyncio.to_thread` - это классический способ превратить блокирующую операцию ввода-вывода в не блокирующую, асинхронную. В частности, библиотека `aioshutil`, которую мы когда-то упоминали, просто оборачивает вызовы оригинального `shutil` в поток. Вот и получаем асинхронность.

Еще раз отдельно выделим, что `FastAPI` из-под коробки поддерживает асинхронность! На семинаре покажем более полезные примеры. Будем интегрироваться с БД. Это, пожалуй, наиболее частое применение асинхронности.

**Пример.** Поддержка `enum`-классов.

In [ ]:
from enum import StrEnum, auto
from http import HTTPStatus

import uvicorn
from fastapi import FastAPI

app = FastAPI()


# Имитируем базу данных
db: dict[str, int] = {}


class Seminarist(StrEnum):
    """Семинарист."""

    MUZAFFAR = auto()
    FEDYA = auto()
    LESIA = auto()
    OLEZHA = auto()
    SASHA = auto()


@app.get("/api/v1/scores")
async def get_scores() -> dict[str, int]:
    """Получить результаты голосования."""
    return db


@app.post("/api/v1/vote", status_code=HTTPStatus.ACCEPTED)
async def vote(seminarist: Seminarist) -> None:
    """Проголосовать за семинариста."""
    db[seminarist] = db.get(seminarist, 0) + 1


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

Что хорошего в этом примере:

1. В `Swagger` будут отображаться только значения из `enum`-класса;
2. В спецификации от `OpenAPI` значения `seminarist` будут ограничены `enum`-классом;
3. Любое значение, отличное от `enum`-класса, не пройдет валидацию на стороне сервера;
4. Можно задать возвращаемый по умолчанию HTTP-код (взгляните на `POST`-метод).

**Пример.** Давайте спрячем аргументы в адрес страницы.

In [ ]:
from enum import StrEnum, auto
from http import HTTPStatus

import uvicorn
from fastapi import FastAPI

app = FastAPI()


# Имитируем базу данных
db: dict[str, int] = {}


class Seminarist(StrEnum):
    """Семинарист."""

    MUZAFFAR = auto()
    FEDYA = auto()
    LESIA = auto()
    OLEZHA = auto()
    SASHA = auto()


@app.get("/api/v1/scores")
async def get_scores() -> dict[str, int]:
    """Получить результаты голосования."""
    return db


@app.post("/api/v1/vote/{seminarist}/{n}", status_code=HTTPStatus.ACCEPTED)
async def vote(seminarist: Seminarist, n: int) -> None:
    """Проголосовать за семинариста."""
    db[seminarist] = db.get(seminarist, 0) + n


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

В коде выше есть проблема. Она называется "отрицательные числа". Попобуйте передать отрицательное число. Что произойдет с числом голосов? А если ноль? Что-то изменится?

Отнимать голоса (или не повышать их количество) мы не хотим. Давайте только прибавлять! Как решить проблему? Нужно научиться валидировать числа относительно нуля. Предостерегаем от добавления `if`-блока. Сделаем по-умному. `FastAPI` прекрасно интегрирован с `pydantic`.

In [ ]:
from enum import StrEnum, auto
from http import HTTPStatus

import uvicorn
from fastapi import FastAPI
from pydantic import PositiveInt  # (!)

app = FastAPI()


# Имитируем базу данных
db: dict[str, int] = {}


class Seminarist(StrEnum):
    """Семинарист."""

    MUZAFFAR = auto()
    FEDYA = auto()
    LESIA = auto()
    OLEZHA = auto()
    SASHA = auto()


@app.get("/api/v1/scores")
async def get_scores() -> dict[str, int]:
    """Получить результаты голосования."""
    return db


@app.post("/api/v1/vote/{seminarist}/{n}", status_code=HTTPStatus.ACCEPTED)
async def vote(seminarist: Seminarist, n: PositiveInt) -> None:
    """Проголосовать за семинариста."""
    db[seminarist] = db.get(seminarist, 0) + n


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

Теперь `FastAPI` будет ругаться на всех, кто попробует приуменьшить заслуги наших семинаристов.

**Пример.** Давайте сломаем спецификацию, которую генерирует `FastAPI`. Попробуем обмануть пользователя, передав не то, что ожидалось.

In [ ]:
from random import randint

import uvicorn
from fastapi import FastAPI
from pydantic import PositiveInt

app = FastAPI()


@app.get("/api/v1/randint")
async def get_random_integer() -> PositiveInt:
    """Получить случайное положительное целое число."""
    return -1 * randint(1, 100)


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

Прелесть `FastAPI` в том, что он защищает не только разработчика от ошибок пользователя, но и пользователя от ошибок разработчика. А нужна была лишь аннотация типов. типов. Давайте рассмотрим еще несколько примеров, как, используя аннотации типов, можно уменьшить количество кода, а значит, упростить себе жизнь.

**Пример.** Напишем сервер, который занимается переписью населения.

In [ ]:
from http import HTTPStatus
from typing import Self

import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field, PositiveFloat, PositiveInt, field_validator

app = FastAPI()


# Имитируем базу данных
db: dict[str, int] = {}


class Citizen(BaseModel):
    """Сущность гражданина."""

    name: str = Field(min_length=2, max_length=32)
    age: PositiveInt = Field(ge=18, le=100)

    @field_validator("name")
    @classmethod
    def ensure_name(cls: type[Self], name: str) -> str:
        """Проверить валидность имени."""
        if not name.isalpha():
            detail = "The name must consist from letters only"
            raise ValueError(detail)

        if not name[0].isupper():
            detail = "The name must start with the capital letter"
            raise ValueError(detail)

        if name != name.lower().capitalize():
            detail = "Only the first letter must be capitalized"
            raise ValueError(detail)

        return name


@app.post("/api/v1/register", status_code=HTTPStatus.ACCEPTED)
async def register(citizen: Citizen) -> None:
    """Зарегистрировать гражданина."""
    if citizen.name in db:
        detail = f"{citizen.name} has already been registered"
        raise HTTPException(HTTPStatus.CONFLICT, detail)

    db[citizen.name] = citizen.age


@app.get("/api/v1/statistics/age/mean")
async def get_mean_age() -> PositiveFloat:
    """Получить средний возраст участника."""
    if not db:
        detail = "The database is empty - try again later"
        raise HTTPException(HTTPStatus.TOO_EARLY, detail)

    return sum(db.values()) / len(db)


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

На сей раз происходит много чего интересного. Во-первых, мы объявили `pydantic`-модель, которая содержит некоторые персональные данные гражданина. Давайте быстренько по ней пробежимся:

Базово:
1. Имя - строка, не меньше 2 и не больше 32 по длине;
2. Возраст - положительное число, не меньше 18 и не больше 100.

Дополнительно:
1. Имя состоит из букв;
2. Имя начинается с заглавной буквы;
3. Имя содержит лишь одну заглавную букву.

Попробуйте поделать разные запросы, которые будут нарушать эти правила. Какую ошибку вы получите?

Надеемся, что теперь вы окончательно убедились в том, что с `FastAPI` вы можете обеспечить себя максимальными гарантиями на валидность входных и выходных данных, чем заметно сократить код.

Идем дальше. Ошибки типа `4XX` в терминах пользовательского опыта, как правило, сопоставимы с исключениями. Эту парадигму перенял `FastAPI`. Для этого используется класс `HTTPException`, куда достаточно передать два объекта - код ошибки и ее текст.

**Примечание.** Давайте взглянем на `OpenAPI`-спецификацию. Сможет ли `FastAPI` распознать более сложную структуру? Спойлер, конечно сможет.

```json
{
  "openapi": "3.1.0",
  "info": {
    "title": "FastAPI",
    "version": "0.1.0"
  },
  "paths": {
    "/api/v1/register": {
      "post": {
        "summary": "Register",
        "description": "Зарегистрировать гражданина.",
        "operationId": "register_api_v1_register_post",
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/Citizen"
              }
            }
          },
          "required": true
        },
        "responses": {
          "202": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {

                }
              }
            }
          },
          "422": {
            "description": "Validation Error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/HTTPValidationError"
                }
              }
            }
          }
        }
      }
    },
    "/api/v1/statistics/age/mean": {
      "get": {
        "summary": "Get Mean Age",
        "description": "Получить средний возраст участника.",
        "operationId": "get_mean_age_api_v1_statistics_age_mean_get",
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {
                  "type": "number",
                  "exclusiveMinimum": 0,
                  "title": "Response Get Mean Age Api V1 Statistics Age Mean Get"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Citizen": {
        "properties": {
          "name": {
            "type": "string",
            "maxLength": 32,
            "minLength": 2,
            "title": "Name"
          },
          "age": {
            "type": "integer",
            "maximum": 100,
            "minimum": 18,
            "exclusiveMinimum": 0,
            "title": "Age"
          }
        },
        "type": "object",
        "required": [
          "name",
          "age"
        ],
        "title": "Citizen",
        "description": "Сущность гражданина."
      },
      "HTTPValidationError": {
        "properties": {
          "detail": {
            "items": {
              "$ref": "#/components/schemas/ValidationError"
            },
            "type": "array",
            "title": "Detail"
          }
        },
        "type": "object",
        "title": "HTTPValidationError"
      },
      "ValidationError": {
        "properties": {
          "loc": {
            "items": {
              "anyOf": [
                {
                  "type": "string"
                },
                {
                  "type": "integer"
                }
              ]
            },
            "type": "array",
            "title": "Location"
          },
          "msg": {
            "type": "string",
            "title": "Message"
          },
          "type": {
            "type": "string",
            "title": "Error Type"
          }
        },
        "type": "object",
        "required": [
          "loc",
          "msg",
          "type"
        ],
        "title": "ValidationError"
      }
    }
  }
}
```

Обратите внимание, как все, что было описано на уровне типов, было перенесно в спецификцию (ограничения на возраст и длину имени). `FastAPI` формирует спецификацию настолько точно, насколько это вообще возможно.

**Примечание.** Кстати говоря, `FastAPI` поддерживает еще и вложенные `pydantic`-модели. На семинаре попробуем и такое.

**Пример.** Давайте теперь представим, что нам нужна какая-то абстрактная ручка, в которую в качестве аргумента можно передавать только буквы. Получается, достаточно ограничить минимальную и максимальную единицей. В `pydantic` таких типов нет. Как быть? Ответ - пользоваться возможностями `FastAPI`.

In [ ]:
from typing import Annotated

import uvicorn
from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/api/v1/ascii")
async def to_ascii(letter: Annotated[str, Query(min_length=1, max_length=1)]) -> int:
    """Получить `ASCII`-код."""
    return ord(letter)


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

Итак, у нас есть `query`-параметр `letter`. `FastAPI` распознает его как `str`, к которому через `typing.Annotated` добавлена дополнительная информация, а именно правила валидации. В этих правилах мы как раз-таки и описали, что нам нужна однобуквенная строка.

**Пример.** Давайте отобразим параметр `letter` в `path`-аргумент.

In [ ]:
from typing import Annotated

import uvicorn
from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/api/v1/ascii/{letter}")
async def to_ascii(letter: Annotated[str, Query(min_length=1, max_length=1)]) -> int:
    """Получить `ASCII`-код."""
    return ord(letter)


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

Такой код работать не будет, потому что `fastapi.Query`, как можно понять, применяется только для `query`-параметров. Достаточно заменить на `fastapi.Path`. По сути это то же самое, но применимо к сущностям другого уровня.

In [ ]:
from typing import Annotated

import uvicorn
from fastapi import FastAPI, Path

app = FastAPI()


@app.get("/api/v1/ascii/{letter}")
async def to_ascii(letter: Annotated[str, Path(min_length=1, max_length=1)]) -> int:
    """Получить `ASCII`-код."""
    return ord(letter)


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

Согласитесь, что это не особо-то и удобно... На помощь снова спешит `pydantic`! На самом деле, `pydantic.Field`, который мы используем для описания моделей можно использовть и для классической аннотации.

In [ ]:
from typing import Annotated

import uvicorn
from fastapi import FastAPI
from pydantic import Field

app = FastAPI()


@app.get("/api/v1/ascii/{letter}")
async def to_ascii(letter: Annotated[str, Field(min_length=1, max_length=1)]) -> int:
    """Получить `ASCII`-код."""
    return ord(letter)


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

Причем `pydantic.Field`, в отличие от `fastapi.Query` и `fastapi.Path`, работет в обоих случаях.

In [ ]:
from typing import Annotated

import uvicorn
from fastapi import FastAPI
from pydantic import Field

app = FastAPI()


@app.get("/api/v1/ascii")
async def to_ascii(letter: Annotated[str, Field(min_length=1, max_length=1)]) -> int:
    """Получить `ASCII`-код."""
    return ord(letter)


def main() -> None:
    """Запустить сервер."""
    uvicorn.run(app)


if __name__ == "__main__":
    main()

Из рассказа выше можно четко отследить, за счет чего, собственно, `FastAPI` получил свое название. Вся валидация лежит на аннотациях. Вы буквально занимаетесь написанием API, а не проверкой предикатов. Более того, `FastAPI` просто идеально интегрирован с `Pydantic`, благодаря чему мед становится еще слаще. Связка этих библиотек - это в чистом виде дзен Python. Явное лучше, чем не явное. Простое лучше, чем сложное.

На этом с `pydantic` закончим. Давайте вернемся к самой библиотеке. В примерах выше вы наверняка заметили, что мы везде пишем `/api/v1`. А вдруг мы захотим изменить префикс на что-то другое? Значит, по-хорошему должно быть единое место для изменений.

В `FastAPI` за это отвечает сущность `Router`. Она объединяет ручки и прочие роутеры в единое целое. Это, в целом, способ сокращения кода и уменьшения зависимостей между элементами API.

In [ ]:
import uvicorn
from fastapi import APIRouter, FastAPI

router = APIRouter(prefix="/api/v1")


@router.get("/echo")
async def to_ascii(text: str) -> str:
    """Эхо-метод."""
    return text


def main() -> None:
    """Запустить сервер."""
    app = FastAPI()
    app.include_router(router)

    uvicorn.run(app)


if __name__ == "__main__":
    main()

Пока не интересно. Давайте добавим логики, чтобы стало понятнее, зачем пользоваться роутерами.

**Пример.** Вернемся к переписи населения. Добавим еще несколько методов статистики по возрасту.

In [ ]:
from http import HTTPStatus
from typing import Self

import uvicorn
from fastapi import APIRouter, FastAPI, HTTPException
from pydantic import BaseModel, Field, PositiveFloat, PositiveInt, field_validator

v1 = APIRouter(prefix="/api/v1")

registration = APIRouter(prefix="/register")
statistics = APIRouter(prefix="/statistics")


# Имитируем базу данных
db: dict[str, int] = {}


class Citizen(BaseModel):
    """Сущность гражданина."""

    name: str = Field(min_length=2, max_length=32)
    age: PositiveInt = Field(ge=18, le=100)

    @field_validator("name")
    @classmethod
    def ensure_name(cls: type[Self], name: str) -> str:
        """Проверить валидность имени."""
        if not name.isalpha():
            detail = "The name must consist from letters only"
            raise ValueError(detail)

        if not name[0].isupper():
            detail = "The name must start with the capital letter"
            raise ValueError(detail)

        if name != name.lower().capitalize():
            detail = "Only the first letter must be capitalized"
            raise ValueError(detail)

        return name


@registration.post("", status_code=HTTPStatus.ACCEPTED)
async def register(citizen: Citizen) -> None:
    """Зарегистрировать гражданина."""
    if citizen.name in db:
        detail = f"{citizen.name} has already been registered"
        raise HTTPException(HTTPStatus.CONFLICT, detail)

    db[citizen.name] = citizen.age


@statistics.get("/age/mean")
async def get_mean_age() -> PositiveFloat:
    """Получить средний возраст участника."""
    if not db:
        detail = "The database is empty - try again later"
        raise HTTPException(HTTPStatus.TOO_EARLY, detail)

    return sum(db.values()) / len(db)


@statistics.get("/age/max")
async def get_max_age() -> PositiveInt:
    """Получить максимальный возраст участника."""
    if not db:
        detail = "The database is empty - try again later"
        raise HTTPException(HTTPStatus.TOO_EARLY, detail)

    return max(db.values())


@statistics.get("/age/min")
async def get_min_age() -> PositiveInt:
    """Получить минимальный возраст участника."""
    if not db:
        detail = "The database is empty - try again later"
        raise HTTPException(HTTPStatus.TOO_EARLY, detail)

    return min(db.values())


def main() -> None:
    """Запустить сервер."""
    v1.include_router(statistics)
    v1.include_router(registration)

    app = FastAPI()
    app.include_router(v1)

    uvicorn.run(app)


if __name__ == "__main__":
    main()

По-хорошему на каждый роутер надо заводить отдельный файл, чтобы не перемешивать логику. Это одно из применений роутеров - разделение логики. Но мы работаем в `Jupyter Notebook`, поэтому такой возможности нет. В идеальном мире у нас бы было три файла:

* Методы `/register`;
* Методы `/statistics/...`;
* Фабрика `/api/v1/...`.

Кроме того, роутеры используют для упрощения включения / исключения методов. Например, на каком-то этапе жизненного цикла продукта нам потребуется вычленить методы регистрации. Возможно, нашли баг. Код удалять не нужно, а вот убрать метод из публичного API нужно. Тогда достаточно выкинуть роутер из `include_router`. Своего рода, `Feature-Toggle`.

На этом не все. Можно заметить, что в `Swagger` методы налепились друг на друга - идут подряд без какой-либо логики. Хочется иметь какую-то разбивку. За это отвечают теги роутеров. Давайте добавим их и снова взглянем на UI.

In [ ]:
from http import HTTPStatus
from typing import Self

import uvicorn
from fastapi import APIRouter, FastAPI, HTTPException
from pydantic import BaseModel, Field, PositiveFloat, PositiveInt, field_validator

v1 = APIRouter(prefix="/api/v1")

registration = APIRouter(prefix="/register", tags=["register"])
statistics = APIRouter(prefix="/statistics", tags=["statistics"])


# Имитируем базу данных
db: dict[str, int] = {}


class Citizen(BaseModel):
    """Сущность гражданина."""

    name: str = Field(min_length=2, max_length=32)
    age: PositiveInt = Field(ge=18, le=100)

    @field_validator("name")
    @classmethod
    def ensure_name(cls: type[Self], name: str) -> str:
        """Проверить валидность имени."""
        if not name.isalpha():
            detail = "The name must consist from letters only"
            raise ValueError(detail)

        if not name[0].isupper():
            detail = "The name must start with the capital letter"
            raise ValueError(detail)

        if name != name.lower().capitalize():
            detail = "Only the first letter must be capitalized"
            raise ValueError(detail)

        return name


@registration.post("", status_code=HTTPStatus.ACCEPTED)
async def register(citizen: Citizen) -> None:
    """Зарегистрировать гражданина."""
    if citizen.name in db:
        detail = f"{citizen.name} has already been registered"
        raise HTTPException(HTTPStatus.CONFLICT, detail)

    db[citizen.name] = citizen.age


@statistics.get("/age/mean")
async def get_mean_age() -> PositiveFloat:
    """Получить средний возраст участника."""
    if not db:
        detail = "The database is empty - try again later"
        raise HTTPException(HTTPStatus.TOO_EARLY, detail)

    return sum(db.values()) / len(db)


@statistics.get("/age/max")
async def get_max_age() -> PositiveInt:
    """Получить максимальный возраст участника."""
    if not db:
        detail = "The database is empty - try again later"
        raise HTTPException(HTTPStatus.TOO_EARLY, detail)

    return max(db.values())


@statistics.get("/age/min")
async def get_min_age() -> PositiveInt:
    """Получить минимальный возраст участника."""
    if not db:
        detail = "The database is empty - try again later"
        raise HTTPException(HTTPStatus.TOO_EARLY, detail)

    return min(db.values())


def main() -> None:
    """Запустить сервер."""
    v1.include_router(statistics)
    v1.include_router(registration)

    app = FastAPI()
    app.include_router(v1)

    uvicorn.run(app)


if __name__ == "__main__":
    main()

Ура, все симпатично! Никогда не пренебрегайте красотой :)

**Спойлер.** На семинаре сделаем уклон на практику:

1. Инъекция зависимостей;
2. Интеграция с БД;
3. Сборка автоклиента;
4. Политика повторных запросов.

## Птица дня

![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ac/6371_Pantanal_toucan_JF.jpg/1280px-6371_Pantanal_toucan_JF.jpg)

Сегодня у нас туканы!

Они так называются, потому что люди слышали это, когда они кричат)

Обитают в Южной Америке, являются ДЯТЛАМИ (да-да, это по сути дятел, просто вот такой у него интересный клювик)

Что интересного в тукане? Клюв, конечно же!

Выглядит громоздким (составляет почти половину туловища), но при этом он полый (то есть легкий), но при этом достаточно крепкий

Вот примерно как он выглядит:

![](https://habrastorage.org/r/w1560/getpro/geektimes/post_images/015/4bb/3ca/0154bb3cab3df71d551979592bdcc953.jpg)

По сути - костная ткань и мембраны, что и делает его одновременно и легким, и прочным (а еще обратите внимание на зазубрины - они нужны, чтобы было удобно еду держать в клюве)